In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [3]:
dataObject = DataProcess()

# static measurement jacobian
static_ind = [0,1,2,4,5,6,8,9,10]
static_Nt = 22
static_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(static_Nt)+'_scale.csv')
num_static = len(static_ind)

static_Q = []
for i in static_ind:
    static_Q.append(dataObject.split_jaco(static_jaco, i,static_Nt))

    
    
dynamic_ind = [3,7,11,12,13]
dynamic_Nt = 5
num_dynamic = len(dynamic_ind)
dynamic_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(dynamic_Nt)+'_scale.csv')

dynamic_Q = []
for i in dynamic_ind:
    dynamic_Q.append(dataObject.split_jaco(dynamic_jaco, i, dynamic_Nt))
    
print("static Q shape:", np.shape(static_Q))
print("dynamic Q shape:", np.shape(dynamic_Q))

jacobian shape: (308, 5)
jacobian shape: (70, 5)
static Q shape: (9, 22, 5)
dynamic Q shape: (5, 5, 5)


In [4]:
Q = []
for i in range(len(static_ind)):
    Q.append(static_Q[i])
for j in range(len(dynamic_ind)):
    Q.append(dynamic_Q[j])

In [5]:
print(np.shape(Q[1]))
print(Q[13])

(22, 5)
[[-0.2369589708548623, -0.0160502930881647, 0.0146807209797872, -0.1763506246058283, 0.2949746088336674], [-0.2369521215057689, -0.0160502930881647, 0.0146807209797872, -0.1763506246058283, 0.2949746088336674], [-0.0695822811896183, -0.0134711387931862, 0.0603797016684929, -0.053746313047065, 0.0260785419601256], [0.1039139858946325, -0.0103535467732923, 0.1023829539095281, 0.0693874136743377, -0.2447841359458552], [0.1671865305544889, -0.0092458200384041, 0.1147703544223888, 0.1104832525762961, -0.3385565785691546]]


In [6]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]
static_idx_upd = [0,1,2,3,4,5,6,7,8]
dynamic_idx_upd = [9,10,11,12,13]

num_param = 5

num_total = num_static + num_dynamic*dynamic_Nt

In [7]:
calculator = MeasurementOptimizer(Q, static_idx_upd, dynamic_idx_upd, num_param, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

[[-0.21932015 -0.00616276  0.10081667 -0.27407024  0.28975981]]
9 0
[[-5.39636671e-01 -2.25417105e-03 -4.98629483e-02  1.93516046e+00
   5.51328212e-01]
 [-3.34076041e-01  1.02165297e-01  1.01694484e-01  2.34038634e+00
   2.21939518e-01]
 [-3.49466114e-01  1.12066203e-01  2.55089661e-01  2.00276080e+00
   1.80977048e-01]
 [-4.67879566e-01  7.89891671e-02  3.01235488e-01  1.68523396e+00
   3.70279801e-01]
 [-3.48707281e-01  1.54449377e-01  3.67953350e-01  2.01685355e+00
   1.88922149e-01]
 [-2.14986754e-01  1.72936751e-01  4.17263190e-01  2.28271100e+00
  -1.28215863e-02]
 [-5.97343517e-02  1.78739705e-01  4.59233947e-01  2.04310988e+00
  -2.10902012e-01]
 [-2.29570567e-01  8.78631069e-02  3.93229773e-01  9.57658275e-01
   8.72533926e-02]
 [-1.23428861e-01  2.30056296e-02  2.48874634e-01  1.71017990e-02
   1.12349447e-01]
 [ 1.27009602e-01  6.21730907e-03 -4.04303820e-02  2.08631027e-02
  -5.48782925e-02]
 [ 6.22871119e-01  1.00771557e-01 -3.70593992e-01  1.91461282e+00
  -7.30592543e-0

[[-0.03225359 -0.00495542  0.14347982  0.03163582  0.03161333]]
13 2
[[-2.90613562e-01 -1.55715906e-02  2.06112091e-01 -6.01160990e-01
   3.50004430e-01]
 [-2.63964241e-01 -1.53825031e-02  2.03053876e-01 -5.64631064e-01
   3.20941995e-01]
 [-1.61676149e-01 -7.03061730e-03  1.80994286e-01 -2.03898951e-01
   1.78374884e-01]
 [-6.58808242e-02 -1.45800532e-03  1.55639787e-01  1.27834339e-01
   7.42808320e-03]
 [-5.63202452e-03  4.36385837e-03  1.17803194e-01  2.88349054e-01
  -7.24461002e-02]
 [ 9.77596024e-03  6.09143059e-03  7.63864044e-02  2.97639333e-01
  -8.49654470e-02]
 [ 1.27380402e-02  3.44967497e-03  2.53200137e-02  1.50557244e-01
  -3.53150690e-02]
 [-2.36003052e-02 -3.75049600e-03 -2.47274944e-02 -1.78082486e-01
   5.16399594e-02]
 [-1.43053893e-01 -1.96807278e-02 -6.14401819e-02 -6.54445097e-01
   2.41729442e-01]
 [-3.74060883e-01 -3.77682761e-02 -4.45267325e-02 -1.25744446e+00
   5.55933138e-01]
 [-6.46554222e-01 -6.20472683e-02  1.31984471e-02 -1.85192394e+00
   9.21790871e-

   0.00000000e+00]]
[[ 0.58302945  0.03357465  0.0934825   2.72735851 -0.47682313]]
14 6
[[ 9.44076687e-02  1.41635099e-03  1.55045275e-01  2.17789567e-02
   2.42861287e-14]
 [ 9.42891193e-02 -3.48933529e-03  1.64288476e-01 -4.61710352e-02
   2.42861287e-14]
 [ 7.35260371e-02  6.39687934e-04  1.40487871e-01 -1.73225249e-01
   2.15105711e-14]
 [ 5.19034717e-02  9.17733307e-03  1.09123875e-01 -2.46198438e-01
  -1.90125693e-13]
 [ 2.94214230e-02  1.72645136e-02  7.49129842e-02 -2.19758237e-01
  -1.97064587e-13]
 [ 1.14484858e-02  1.54626554e-02  3.96582322e-02 -1.16716493e-01
  -2.04003481e-13]
 [ 2.96373523e-05 -4.82801997e-03  9.48964841e-05  2.83205891e-02
   6.23416249e-17]
 [-5.96557562e-03 -3.19942815e-02 -1.23846994e-02  1.68616974e-03
   7.28583860e-15]
 [-1.73166815e-03 -5.63490228e-02 -1.24994250e-02 -7.33572116e-02
   2.49800181e-14]
 [-3.73388300e-02 -8.84026131e-02 -8.30202599e-02  1.96081236e-01
   5.27355937e-14]
 [-7.03971795e-02 -1.09417703e-01 -1.45379997e-01  4.22261925

[[-2.90613562e-01 -1.55715906e-02  2.06112091e-01 -6.01160990e-01
   3.50004430e-01]
 [-2.63964241e-01 -1.53825031e-02  2.03053876e-01 -5.64631064e-01
   3.20941995e-01]
 [-1.61676149e-01 -7.03061730e-03  1.80994286e-01 -2.03898951e-01
   1.78374884e-01]
 [-6.58808242e-02 -1.45800532e-03  1.55639787e-01  1.27834339e-01
   7.42808320e-03]
 [-5.63202452e-03  4.36385837e-03  1.17803194e-01  2.88349054e-01
  -7.24461002e-02]
 [ 9.77596024e-03  6.09143059e-03  7.63864044e-02  2.97639333e-01
  -8.49654470e-02]
 [ 1.27380402e-02  3.44967497e-03  2.53200137e-02  1.50557244e-01
  -3.53150690e-02]
 [-2.36003052e-02 -3.75049600e-03 -2.47274944e-02 -1.78082486e-01
   5.16399594e-02]
 [-1.43053893e-01 -1.96807278e-02 -6.14401819e-02 -6.54445097e-01
   2.41729442e-01]
 [-3.74060883e-01 -3.77682761e-02 -4.45267325e-02 -1.25744446e+00
   5.55933138e-01]
 [-6.46554222e-01 -6.20472683e-02  1.31984471e-02 -1.85192394e+00
   9.21790871e-01]
 [-7.13713236e-01 -7.33612645e-02  4.92418072e-02 -2.03426877e+00

  -1.82033887e+00]]
[[ 0.01214436 -0.01071182  0.04784475 -0.65117437  0.00094151]]
23 8
[[-6.93196458e-01 -3.01364300e-02 -2.35728637e-01  8.28528399e-02
   1.02120451e+00]
 [-4.41212229e-01 -2.00669297e-02 -1.49582521e-01  1.44351252e-01
   6.29576143e-01]
 [-1.06222838e-02  6.87580395e-03 -3.19548719e-02  2.94217918e-01
  -1.65985340e-02]
 [ 2.69631679e-01  1.89716388e-02  2.18719788e-02  3.63258179e-01
  -3.85135439e-01]
 [ 3.70142773e-01  2.22745240e-02  3.07890617e-02  3.60924336e-01
  -4.94885431e-01]
 [ 3.18407395e-01  1.74813503e-02  2.12751063e-02  2.85201269e-01
  -4.09820510e-01]
 [ 1.56779110e-01  9.55789023e-03  1.22853931e-02  1.09301174e-01
  -1.99862584e-01]
 [-2.11995762e-01 -8.07904644e-03 -2.98206570e-02 -1.06842860e-01
   2.76777522e-01]
 [-7.19331528e-01 -1.70294384e-02 -8.86282738e-02 -1.85978524e-01
   9.21709408e-01]
 [-4.24688468e-01 -2.07437150e-02  7.22125726e-02 -1.99936320e-01
   4.83605146e-01]
 [ 1.29623359e-02 -1.39127620e-02  2.42727844e-01 -2.34124526

   1.46070698e-02]]
[[-0.08334115  0.00597289 -0.04542772  0.29516975 -0.03495748]]
24 7
[[ 1.03280376e+00  4.30423210e-02  7.58693869e-01  2.26756931e-01
  -1.44976540e+00]
 [ 1.05680171e+00  4.92148411e-02  7.74692915e-01  1.23243069e-01
  -1.49202170e+00]
 [ 8.82073836e-01  3.01240542e-02  6.47271685e-01 -1.07672469e-02
  -1.26637243e+00]
 [ 5.20476749e-01  1.25355167e-02  3.93266176e-01 -3.95228885e-02
  -7.47659124e-01]
 [ 2.17711445e-01  3.68374559e-04  1.71855042e-01 -2.70793037e-03
  -3.10852466e-01]
 [ 4.96559380e-02 -1.46810739e-03  4.99691801e-02 -8.54287556e-03
  -6.30984915e-02]
 [-7.50506820e-03  5.28482721e-03 -6.55551389e-03 -1.88588008e-02
   2.23906575e-03]
 [-4.82860079e-02  1.59838620e-02 -5.25806095e-02 -4.48098001e-02
   4.46194549e-02]
 [-9.26986055e-02  1.97862062e-02 -9.22549387e-02 -1.75370729e-02
   1.11305847e-01]
 [-1.22573827e-01  2.04133414e-02 -9.95863381e-02 -8.99419729e-03
   1.70584438e-01]
 [-1.27065049e-01  2.03576360e-02 -9.56745821e-02  3.69439072

   1.46070698e-02]]
[[-0.23695212 -0.01605029  0.01468072 -0.17635062  0.29497461]]
30 7
[[ 1.03280376e+00  4.30423210e-02  7.58693869e-01  2.26756931e-01
  -1.44976540e+00]
 [ 1.05680171e+00  4.92148411e-02  7.74692915e-01  1.23243069e-01
  -1.49202170e+00]
 [ 8.82073836e-01  3.01240542e-02  6.47271685e-01 -1.07672469e-02
  -1.26637243e+00]
 [ 5.20476749e-01  1.25355167e-02  3.93266176e-01 -3.95228885e-02
  -7.47659124e-01]
 [ 2.17711445e-01  3.68374559e-04  1.71855042e-01 -2.70793037e-03
  -3.10852466e-01]
 [ 4.96559380e-02 -1.46810739e-03  4.99691801e-02 -8.54287556e-03
  -6.30984915e-02]
 [-7.50506820e-03  5.28482721e-03 -6.55551389e-03 -1.88588008e-02
   2.23906575e-03]
 [-4.82860079e-02  1.59838620e-02 -5.25806095e-02 -4.48098001e-02
   4.46194549e-02]
 [-9.26986055e-02  1.97862062e-02 -9.22549387e-02 -1.75370729e-02
   1.11305847e-01]
 [-1.22573827e-01  2.04133414e-02 -9.95863381e-02 -8.99419729e-03
   1.70584438e-01]
 [-1.27065049e-01  2.03576360e-02 -9.56745821e-02  3.69439072

In [8]:
print(calculator.Sigma_inv[(9,0)])
print(calculator.Q_dynamic_flatten[0])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-0.539636670963493, -0.0022541710490597, -0.0498629482614828, 1.9351604614661069, 0.5513282116769436], [-0.3340760407153356, 0.1021652968364613, 0.1016944838159672, 2.3403863441379817, 0.221939518269873], [-0.3494661138571784, 0.1120662029219454, 0.2550896605274281, 2.0027607969742065, 0.1809770476947275], [-0.4678795656821666, 0.0789891670526334, 0.3012354877648384, 1.6852339585553322, 0.3702798007923669], [-0.3487072812830348, 0.1544493767900593, 0.3679533495956177, 2.016853552909431, 0.1889221494208309], [-0.2149867536290262, 0.1729367514893193, 0.4172631902483755, 2.2827109973126425, -0.0128215863177127], [-0.0597343516865485, 0.1787397054563655, 0.4592339471516512, 2.0431098799382568, -0.2109020116805193], [-0.2295705671608305, 0.0878631069081999, 0.393229772557104, 0.9576582753596296, 0.0872533926070839], [-0.1234288608689441, 0.0230056295989378, 

In [9]:
print(calculator.dynamic_to_flatten[1])

[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]


### Set up grey-box 

In [11]:
from greybox_generalize import LogDetModel

In [12]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

### Reform the inputs

In [13]:
cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         200, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         200, #des.gas_outlet.T (6)
         600, #ads.T19 (8)
         600, #ads.T23 (9)
         600 #ads.T28 (10)
       ]

for i in range(dynamic_Nt*num_dynamic):
    cost.append(10)


In [14]:
#print(np.shape(ans_y))

init_y = np.random.rand(num_total)

update_y = [] 

for i in init_y:
    if i > 0.5: 
        update_y.append(1)
    else:
        update_y.append(0)

init_cov_y = [[0]*num_total for i in range(num_total)]

for j in range(num_total):
    for k in range(num_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [15]:
mip_option = False
objective = "D"
budget_opt = 5000
sparse_opt = False
fix_opt = False

In [16]:
mod = calculator.continuous_optimization(cost, mixed_integer=mip_option, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, budget=budget_opt,
                                        init_fim = fim_prior)

In [17]:

if not mip_option and objective=="A":
    solver = pyo.SolverFactory('ipopt')
    solver.options['linear_solver'] = "ma57"
    solver.solve(mod, tee=True)

elif mip_option and objective=="A":
    solver = pyo.SolverFactory('mindtpy')
    results = solver.solve(mod, tee=True, integer_tolerance=0.5)

elif objective=="D":  
    solver = pyo.SolverFactory('cyipopt')
    solver.config.options['hessian_approximation'] = 'limited-memory' 
    additional_options={'max_iter':3000}
    for k,v in additional_options.items():
        solver.config.options[k] = v
    results = solver.solve(mod, tee=True)


 Consider M =
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
   logdet =  0.0 


 Consider M =
 [[ 0.97369842  0.00835158  0.02395997  0.00868845 -0.11028045]
 [ 0.00835158  0.95452412 -0.00278841  0.05542268 -0.01010292]
 [ 0.02395997 -0.00278841  0.90944598  0.16927237  0.03558703]
 [ 0.00868845  0.05542268  0.16927237  1.12539039 -0.21058436]
 [-0.11028045 -0.01010292  0.03558703 -0.21058436  1.05266558]]
   logdet =  -0.0865214428465619 


 Consider M =
 [[ 0.96036615  0.01529935  0.0828237  -0.31781003 -0.36618749]
 [ 0.01529935  0.87720761  0.00529808  0.13747386 -0.02249035]
 [ 0.0828237   0.00529808  0.77899203  0.42704329  0.12997022]
 [-0.31781003  0.13747386  0.42704329  2.26841271 -0.41096957]
 [-0.36618749 -0.02249035  0.12997022 -0.41096957  1.22623344]]
   logdet =  0.027673117848244666 


 Consider M =
 [[ 1.83490784  0.05578015 -0.35268905  0.62101107 -1.48697421]
 [ 0.05578015  0.66189509  0.03416075  0.47533204 -0.0862805


 Consider M =
 [[ 7.63826206e+01  2.71240316e-01 -5.25822033e+01  1.34902921e+02
  -1.31985084e+01]
 [ 2.71240316e-01  4.03916239e-01 -7.50844309e-02  4.24399466e+00
  -8.09819772e-01]
 [-5.25822033e+01 -7.50844309e-02  4.60709734e+01 -1.31406597e+02
   5.34685990e+00]
 [ 1.34902921e+02  4.24399466e+00 -1.31406597e+02  5.56206899e+02
  -2.34114577e+01]
 [-1.31985084e+01 -8.09819772e-01  5.34685990e+00 -2.34114577e+01
   1.58615345e+01]]
   logdet =  12.877729401891097 


 Consider M =
 [[ 1.31369101e+02  2.88481086e-01 -9.53073720e+01  2.46329191e+02
  -1.62835435e+01]
 [ 2.88481086e-01  4.14831390e-01 -9.04860973e-02  4.43282024e+00
  -8.40210705e-01]
 [-9.53073720e+01 -9.04860973e-02  7.95648457e+01 -2.18443836e+02
   7.61117600e+00]
 [ 2.46329191e+02  4.43282024e+00 -2.18443836e+02  8.00654977e+02
  -2.96947191e+01]
 [-1.62835435e+01 -8.40210705e-01  7.61117600e+00 -2.96947191e+01
   1.64775805e+01]]
   logdet =  13.688255638812834 


 Consider M =
 [[ 2.16485539e+02  3.19244680e-0

In [18]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        fim_result[i,j] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 1.08457446e+03  4.94348488e-01 -8.37629130e+02  2.18047684e+03
  -6.82328657e+01]
 [ 4.94348488e-01  4.85264641e-01 -1.52908006e-01  5.95811589e+00
  -1.03832491e+00]
 [-8.37629130e+02 -1.52908006e-01  6.59898952e+02 -1.71994652e+03
   4.62736400e+01]
 [ 2.18047684e+03  5.95811589e+00 -1.71994652e+03  4.94353145e+03
  -1.31749957e+02]
 [-6.82328657e+01 -1.03832491e+00  4.62736400e+01 -1.31749957e+02
   2.38564179e+01]]
trace: 6712.34654542105
det: 39408531.05615257
Pyomo OF: 17.489492877259178
Log_det: 17.489492875097998


In [19]:
# problem size calculator
if True:
    nr = num_total
    npa = 5
    num_var = (nr*nr+nr)/2 + npa*npa + 1
    num_equal = npa*npa+1
    num_ineq = 1.5*(nr**2-nr)+1

    print(num_var, num_equal, num_ineq)

621.0 26 1684.0


In [ ]:
import numpy as np

ans_y = np.zeros((num_total,num_total))

pyomo_y = np.zeros((num_total, num_total))

for i in range(num_total):
    for j in range(i, num_total):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = cov 
        ans_y[j,i] = pyo.value(mod.cov_y[j,i])

for i in range(num_total):
    for j in range(num_total):
        pyomo_y[i,j] = pyo.value(mod.cov_y[i,j])
        
print("Cost for this stragy:", pyo.value(mod.cost))
print("Cost if no budget:", sum(cost))

In [ ]:
figure = plt.figure()
axes = figure.add_subplot(111)

caxes = axes.matshow(ans_y, interpolation='nearest')
figure.colorbar(caxes)

plt.show()

print(np.shape(ans_y))

In [ ]:
for i in range(len(ans_y)):
    for j in range(len(ans_y[0])):
        if ans_y[i][j] < 0.05:
            ans_y[i][j] = int(0)
        elif ans_y[i][j] > 0.95:
            ans_y[i][j] = int(1)
        else: 
            ans_y[i][j] = round(ans_y[i][j], 2)
print(ans_y)

In [ ]:
import pickle

store = False

if store:
    file = open('Fe23_D_110_3000', 'wb')

    pickle.dump(ans_y, file)

    file.close()

In [ ]:
for c in range(num_static):
    print(static_name[c], ": ", ans_y[c,c])

sol_y = np.asarray([ans_y[i,i] for i in range(num_static, num_total)])

sol_y = np.reshape(sol_y, (num_dynamic, dynamic_Nt))
np.around(sol_y)

for r in range(len(sol_y)):
    print(dynamic_name[r], ": ", sol_y[r])

In [ ]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
#measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
#measure_name = ['Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
measure_name = ['a', 'b', 'c', 'd', 'e']

#time_set = ['500', '1000', '1500', '2000', '2200']
time_set = np.linspace(20, 2200, number_t)
time_set = [str(i) for i in time_set]
print(len(time_set), len(measure_name))
print(np.shape(sol_y))

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, time_set, measure_name, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
def plot_data(budget, a_cri, d_cri):

    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    #plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(A-optimality)', fontweight="bold")
    
    plt.show()
    
    
    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(D-optimality)', fontweight="bold")
    
    plt.show()

In [ ]:
a_set = [65693.94, 66574.63, 67474.485, 67608.817]
d_set = [np.exp(30.13), np.exp(30.719), np.exp(31.1205), np.exp(31.2586)]

plot_data([3000,4000,5000,6400], a_set, d_set)